In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tsfresh as tsf
import random
from torch.utils.data import random_split

df = pd.read_csv(r'C:\\Users\\Jiananyuan\Desktop\\QuickAccess\\DM_homework\dataset\\train.csv')
signals = df['heartbeat_signals'].str.split(',', expand=True)
signals.insert(0, '', df['label'], allow_duplicates=False)
col_names = ['label']
for i in range(0, 205):
    col_names.append('signal_' + str(i))
signals.columns = col_names
signals = pd.DataFrame(signals, dtype=np.float64)

def signal_enhance(_signal, sigma=0.1):
    scalingFactor = np.random.normal(loc=1.0, scale=sigma, size=(1, _signal.shape[1]))
    noise = np.matmul(np.ones((_signal.shape[0], 1)), scalingFactor)
    return _signal * noise

idx_set = signals.query('label==1').index
record_label_1 = signals.iloc[idx_set, :].reset_index(drop=True)
enhanced_signal_label_1 = record_label_1
for i in np.arange(16):
    tmp_enhanced_signal_label_1 = signal_enhance(record_label_1)
    enhanced_signal_label_1 = pd.concat((enhanced_signal_label_1, 
                                         tmp_enhanced_signal_label_1), axis=0).reset_index(drop=True)
enhanced_signal_label_1['label'] = 1

idx_set = signals.query('label==2').index
record_label_2 = signals.iloc[idx_set, :].reset_index(drop=True)
enhanced_signal_label_2 = record_label_2
for i in np.arange(3):
    tmp_enhanced_signal_label_2 = signal_enhance(record_label_2)
    enhanced_signal_label_2 = pd.concat((enhanced_signal_label_2, tmp_enhanced_signal_label_2), axis=0).reset_index(
        drop=True)
enhanced_signal_label_2['label'] = 2

idx_set = signals.query('label==3').index
record_label_3 = signals.iloc[idx_set, :].reset_index(drop=True)
enhanced_signal_label_3 = record_label_3
for i in np.arange(3):
    tmp_enhanced_signal_label_3 = signal_enhance(record_label_3)
    enhanced_signal_label_3 = pd.concat((enhanced_signal_label_3, tmp_enhanced_signal_label_3), axis=0).reset_index(
        drop=True)
enhanced_signal_label_3['label'] = 3

idx_set = signals.query('label==0').index
record_label_0 = signals.iloc[idx_set, :].reset_index(drop=True)

data_train = pd.concat([record_label_0, 
                        enhanced_signal_label_1, 
                        enhanced_signal_label_2, 
                        enhanced_signal_label_3], ignore_index=True)

def remove_last_zero(_series):
    nps = _series.to_numpy()
    zero_begin_idxs = np.arange(nps.shape[0])
    for i in np.arange(nps.shape[1])[::-1]:
        idxs = np.where(nps[zero_begin_idxs, i] <= 1.e-5)[0]
        if idxs.size > 0:
            nps[zero_begin_idxs[idxs], i] = np.nan
            zero_begin_idxs = zero_begin_idxs[idxs]
        else:
            break
    return pd.DataFrame(nps[:, :], index=_series.index, columns=_series.columns[:])

# data_train = remove_last_zero(data_train)

# drop_idx = list(data_train.index)
# drop_idx = random.sample(drop_idx, 3325)
# data_train = data_train.drop(drop_idx)


253325

In [53]:
from sklearn.utils import shuffle

data_train = shuffle(data_train)
net_train_data = data_train.iloc[:, 1:].to_numpy()
net_train_label = data_train.iloc[:, 0].to_numpy()

In [60]:
train_set = data_train.iloc[:200000, :]
test_set = data_train.iloc[200000:, :]

train_x = train_set.iloc[:, 1:].to_numpy()
train_y = train_set.iloc[:, 0].to_numpy()
valid_x = test_set.iloc[:, 1:].to_numpy()
valid_y = test_set.iloc[:, 0].to_numpy()

# train_data = data_train.iloc[:, 1:]
# train_label = data_train.iloc[:, 0]

# train_data = data_train.iloc[:, 1:].stack()
# train_data = train_data.reset_index()
# train_data.rename(columns={"level_0": "id", "level_1": "time", 0: "signals"}, 
#                   inplace=True)
# train_data["signals"] = train_data["signals"].astype(float)

# train_data

print(train_x.shape, train_y.shape, valid_x.shape, valid_y.shape)


(200000, 205) (200000,) (53325, 205) (53325,)


In [16]:
# train_label.values.tolist()
# net_train_data = train_data.values
# net_train_label = train_label.values
# net_train_data

array([[0.9912298 , 0.94353304, 0.7646773 , ...,        nan,        nan,
               nan],
       [0.9714822 , 0.92896875, 0.57293281, ...,        nan,        nan,
               nan],
       [0.97579528, 0.93408847, 0.65963666, ...,        nan,        nan,
               nan],
       ...,
       [0.88023307, 0.89468516, 0.89448266, ...,        nan,        nan,
               nan],
       [0.71712964, 0.68011328, 0.63003724, ...,        nan,        nan,
               nan],
       [0.84084461, 0.5637697 , 0.63639073, ...,        nan,        nan,
               nan]])

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(
                in_channels=1,
                out_channels=64,
                kernel_size=7,
                stride=1,
                padding=3,
            ),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv1d(64, 128, 5, 1, 2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv1d(128, 256, 3, 1, 1), 
            nn.BatchNorm1d(256),
            nn.ReLU(), 
            nn.Conv1d(256, 256, 3, 1, 1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(6400, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(),
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
        )
        
        self.fc3 = nn.Linear(64, 4)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        x = x.view(x.size(0), -1)
#         print(x.size())
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [25]:
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import copy

# 训练网络
def train_net(net, train_loader, valid_loader, EPOCH=20, LR=1e-5):
    # 交叉熵作为损失函数
    loss_fn = nn.CrossEntropyLoss()
    
    # Adam 优化器
    optimizer = torch.optim.Adam(net.parameters(), lr=LR)
    
    # 保留准确率最高的模型
    best_state = copy.deepcopy(net.state_dict())
    best_acc = 0

    for epoch in range(EPOCH):
        net.train()
        train_l = 0
        train_num = 0
        for b_x, b_y in train_loader:
            output = net(b_x)
            loss = loss_fn(output, b_y)
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()

            train_l += loss.cpu().item()
            train_num += len(b_x)
        
        net.eval()
        with torch.no_grad():
            valid_l = 0
            valid_num = 0
            right_num = 0.
            for b_x, b_y in valid_loader:
                output = net(b_x)
                loss = loss_fn(output, b_y)
                valid_l += loss.cpu().item()

                pred_y = torch.argmax(output, dim=1)
                valid_num += len(b_x)
                right_num += (pred_y == b_y).sum().item()
        valid_acc = right_num / valid_num
        if valid_acc >= best_acc:
            best_acc = valid_acc
            best_state = copy.deepcopy(net.state_dict())
        print('Epoch {} | train_loss {:.2e} | valid_loss {:.2e} | acc {:f} | best_acc {:f}'.format(epoch + 1, train_l / train_num, valid_l / valid_num, valid_acc, best_acc))
        
    
    return best_state


In [36]:
!pip3 install imblearn

In [57]:
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from torch.utils.data import TensorDataset
import time
net_states = []
seed = 2023

# 采用 10 折交叉验证，训练 10 个模型
sk = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
for i, (train, valid) in enumerate(sk.split(net_train_data, net_train_label)):
    x_train = net_train_data[train]
    y_train = net_train_label[train]
    x_valid = net_train_data[valid]
    y_valid = net_train_label[valid]
    
    # # 对训练数据进行随机过采样
    x_train, y_train = RandomOverSampler(random_state=seed).fit_resample(x_train, y_train)

    x_train = torch.tensor(x_train, dtype=torch.float).unsqueeze(-2)
    x_valid = torch.tensor(x_valid, dtype=torch.float).unsqueeze(-2)
    y_train = torch.tensor(y_train, dtype=int)
    y_valid = torch.tensor(y_valid, dtype=int)
    
    # train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=512, shuffle=True, num_workers=2)
    # valid_loader = DataLoader(TensorDataset(x_valid, y_valid), batch_size=1024, shuffle=False)
    
    # print('\nTraining {}th net. {}\n'.format(i + 1, time.strftime('%H:%M:%S')))
    # net = Net()
    # s = train_net(net, train_loader, valid_loader, 30)
    # net_states.append(s)
    print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)

# 保存模型参数
torch.save(net_states, 'net_state.pkl') 

torch.Size([257932, 1, 205]) torch.Size([257932]) torch.Size([25333, 1, 205]) torch.Size([25333])
torch.Size([257932, 1, 205]) torch.Size([257932]) torch.Size([25333, 1, 205]) torch.Size([25333])
torch.Size([257932, 1, 205]) torch.Size([257932]) torch.Size([25333, 1, 205]) torch.Size([25333])
torch.Size([257932, 1, 205]) torch.Size([257932]) torch.Size([25333, 1, 205]) torch.Size([25333])
torch.Size([257932, 1, 205]) torch.Size([257932]) torch.Size([25333, 1, 205]) torch.Size([25333])
torch.Size([257936, 1, 205]) torch.Size([257936]) torch.Size([25332, 1, 205]) torch.Size([25332])
torch.Size([257936, 1, 205]) torch.Size([257936]) torch.Size([25332, 1, 205]) torch.Size([25332])
torch.Size([257932, 1, 205]) torch.Size([257932]) torch.Size([25332, 1, 205]) torch.Size([25332])
torch.Size([257932, 1, 205]) torch.Size([257932]) torch.Size([25332, 1, 205]) torch.Size([25332])
torch.Size([257932, 1, 205]) torch.Size([257932]) torch.Size([25332, 1, 205]) torch.Size([25332])


In [61]:
seed = 2023

# x_train, x_label = RandomOverSampler(random_state=seed).fit_resample(x_train, x_label)

train_x = torch.tensor(train_x, dtype=torch.float).unsqueeze(-2)
valid_x = torch.tensor(valid_x, dtype=torch.float).unsqueeze(-2)
train_y = torch.tensor(train_y, dtype=int)
valid_y = torch.tensor(valid_y, dtype=int)
print(train_x.shape, train_y.shape, valid_x.shape, valid_y.shape)

train_loader = DataLoader(TensorDataset(train_x, train_y), batch_size=512, shuffle=True, num_workers=2)
valid_loader = DataLoader(TensorDataset(valid_x, valid_y), batch_size=1024, shuffle=False)

print('\nTraining {}th net. {}\n'.format(i + 1, time.strftime('%H:%M:%S')))
net = Net()
s = train_net(net, train_loader, valid_loader, 30)
torch.save(s, 'net_state.pkl')


torch.Size([200000, 1, 205]) torch.Size([200000]) torch.Size([53325, 1, 205]) torch.Size([53325])

Training 10th net. 13:22:00



In [ ]:
net = Net().to(device)

# 读入模型参数
net_states = torch.load('net_state.pkl')
for s in net_states:
    net.load_state_dict(s)
    net.eval()
    with torch.no_grad():
        cur_pred = []
        for i in range(100):
            b_x = test_x[i * 200: (i + 1) * 200].to(device)
            output = nn.functional.softmax(net(b_x), dim=1).cpu().tolist()
            cur_pred.extend(output)
    test_pred.append(cur_pred)
test_pred = np.array(test_pred)
test_avg = test_pred.mean(axis=0)
test_y_pred = test_avg.argmax(axis=1)
test_y_pred